In [18]:
%pip install tabula-py
import pandas as pd
import tabula
import numpy as np
from IPython.utils import io


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Importation du notebook Calendrier

In [19]:
%pip install tabula-py
try:
    with io.capture_output():
        %run -i "Calendrier.ipynb"
except Exception as e:
    display(e)


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


NameError: name 'df' is not defined

NameError("name 'df' is not defined")

Importation du notebook Informations lieux des compétitions

In [ ]:
try:
    with io.capture_output():
        %run -i "Informations lieux des compétitions.ipynb"
except Exception as e:
    display(e)

Exception("File `'Informations lieux des compétitions.ipynb'` not found.")

Importation du notebook Informations lignes métro

In [ ]:
try:
    with io.capture_output():
        %run -i "Informations lignes métro.ipynb"
except Exception as e:
    display(e)

Exception("File `'Informations lignes métro.ipynb'` not found.")

Programme calculant le nombre de personnes sans les JO

In [ ]:
def affluence_hors_jo(horaire, station, ligne):
    # Obtenir le tableau correspondant à la ligne
    line_table = tableaux[f'ligne numéro[{ligne}]']

    # Extraire l'heure de l'horaire entré par l'utilisateur
    hour, minute = map(int, horaire.split(':'))

    # Construire la chaîne de plage horaire correspondante
    plage_horaire = f"{hour:02d}:00-{(hour + 1) % 24:02d}:00"

    # Vérifier si la plage horaire se trouve dans les colonnes du tableau
    if plage_horaire not in line_table.columns:
        raise ValueError(f"Plage horaire non trouvée dans le tableau: {plage_horaire}")

    # Récupérer le nombre à la station et à la plage horaire donnée
    affluence_horaire = line_table[line_table['Station'] == station][plage_horaire].iloc[0]

    # Récupérer l'indice de la plage horaire suivante
    next_time_range_index = line_table.columns.get_loc(plage_horaire) + 1
    next_time_range_index = next_time_range_index % len(line_table.columns)  # Assurer la circularité

    # Récupérer la plage horaire suivante
    plage_horaire_suivante = line_table.columns[next_time_range_index]

    # Récupérer le nombre à la plage horaire suivante
    affluence_suivante = line_table[line_table['Station'] == station][plage_horaire_suivante].iloc[0]

    # Récupérer la moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction à la station
    moyenne_ligne = line_table[line_table['Station'] == station]['moyenne_NB_VALD_hebdo_période_jo_2022_par_ligne_par_direction'].iloc[0]

    return round(affluence_horaire), round(affluence_suivante), round(moyenne_ligne)

# Exemple d'utilisation
horaire_info = "07:35"
station_info = "hotel de ville"
ligne_info = "1"

try:
    affluence_horaire, affluence_suivante, affluence_ligne = affluence_hors_jo(horaire_info, station_info, ligne_info)
    print(f"Affluence à {horaire_info} à la station {station_info}, sur la ligne numéro {ligne_info} : {affluence_horaire}")
    print(f"Affluence à la plage horaire suivante à la station {station_info}, sur la ligne numéro {ligne_info} : {affluence_suivante}")
    print(f"Affluence sur la ligne {ligne_info} : {affluence_ligne}")
except ValueError as e:
    print(e)


NameError: name 'tableaux' is not defined

Programme calculant le nombre de personnes supplémentaires avec les JO

In [ ]:
def calculer_personnes_supplementaires(date_horaire_entree, numero_ligne, station):

    tableau_debut = pd.DataFrame()  # Initialisation du DataFrame vide
    tableau_fin = pd.DataFrame()    # Initialisation du DataFrame vide

    nb_station = 0 
    nb_ligne = 0 

    cle_tableau = f'ligne numéro[{numero_ligne}]'

    # Extraire l'horaire de l'entrée utilisateur
    heure_entree = date_horaire_entree[-5:]

    # Déterminer la plage horaire à laquelle appartient l'horaire d'entrée
    plage_horaire_entree = f"{heure_entree.split(':')[0]}:00:00 - {(int(heure_entree.split(':')[0])+1)%24:02d}:00:00"

    # Construire la chaîne de date avec la plage horaire
    date_plage_horaire = f"{date_horaire_entree[:10]} {plage_horaire_entree}"

    # Vérifier si la chaîne date_plage_horaire se trouve dans new_horaires_début_df
    if (date_plage_horaire in new_horaires_début_df.values) or (date_plage_horaire in new_horaires_fin_df.values): 
        # Afficher les éléments de la colonne "Lieu" correspondants à la condition dans new_horaires_début_df
        lieux_correspondants_debut = new_horaires_début_df.loc[new_horaires_début_df.eq(date_plage_horaire).any(axis=1), 'Lieu']
        # Afficher les éléments de la colonne "Lieu" correspondants à la condition dans new_horaires_fin_df
        lieux_correspondants_fin = new_horaires_fin_df.loc[new_horaires_fin_df.eq(date_plage_horaire).any(axis=1), 'Lieu']
        
        lignes_correspondantes = []
        
        
        # Renvoyer les vecteurs de la colonne Ligne pour chaque élément de lieux_correspondants_debut
        for lieu in lieux_correspondants_debut:
            ligne_lieu = df[df['Lieu'] == lieu]  # Trouver la ligne correspondant au lieu
            tableau_debut=pd.concat([tableau_debut, ligne_lieu], ignore_index=True)
            lignes_lieu = ligne_lieu['Lignes']
            lignes_correspondantes.extend(lignes_lieu.values)
            

        # Renvoyer les vecteurs de la colonne Ligne pour chaque élément de lieux_correspondants_fin
        for lieu in lieux_correspondants_fin:
            ligne_lieu = df[df['Lieu'] == lieu]  # Trouver la ligne correspondant au lieu
            tableau_fin=pd.concat([tableau_fin, ligne_lieu], ignore_index=True)
            lignes_lieu = ligne_lieu['Lignes']
            lignes_correspondantes.extend(lignes_lieu.values)

        numero_trouve = False

        for sous_liste in lignes_correspondantes:
            if numero_ligne in sous_liste:
                numero_trouve = True
                break  # Arrête la boucle une fois que le numéro est trouvé

            # Vérifier si le numéro de ligne entré est présent dans les vecteurs des lignes correspondantes
        if numero_trouve:
            resultat_debut=0
            resultat_fin=0
            if not tableau_debut.empty :
                tableau_debut = tableau_debut[tableau_debut['Lignes'].apply(lambda x: numero_ligne in x)]
                resultat_debut = tableau_debut['Nombre de personnes par ligne'].sum()
            if not tableau_fin.empty :
                    tableau_fin = tableau_fin[tableau_fin['Lignes'].apply(lambda x: numero_ligne in x)]
                    resultat_fin = tableau_fin['Nombre de personnes par ligne'].sum()
            resultat=resultat_debut+resultat_fin
            nb_ligne=round(resultat)

            resultat_debut_2=0
            resultat_fin_2=0

            if not tableau_debut.empty:
                tableau_provisoire =tableaux[cle_tableau]
                ligne_provisoire = tableau_provisoire[tableau_provisoire['Station'].str.contains(station)]
                valeur = ligne_provisoire['Part du trafic de la station par rapport à toute la ligne'].values[0]
                tableau_debut['Pourcentage fréquentation'] = valeur
                resultat_debut_2=valeur*resultat_debut
            
            if not tableau_fin.empty:
                tableau_provisoire =tableaux[cle_tableau]
                ligne_provisoire = tableau_provisoire[tableau_provisoire['Station'].str.contains(station)]
                valeur = ligne_provisoire['Part de la station dans le trafic annuel de la ligne'].values[0]
                tableau_fin['Pourcentage fréquentation'] = valeur
                resultat_fin_2=valeur*resultat_debut
            
            resultat2=resultat_debut_2+resultat_fin_2
            nb_station=round(resultat2)
            return nb_ligne, nb_station

        else:
            print(0)
    else:
        print(0)
    
    return nb_ligne, nb_station

#exemple d'utilisation 
date_horaire_entree = '2024-07-28 09:20'
numero_ligne = '1'
station = 'bastille'

nb_ligne, nb_station = calculer_personnes_supplementaires(date_horaire_entree, numero_ligne, station)

phrase_station = f"Le nombre de personnes supplémentaires à la station {station} en raison des JO est {nb_station}."
phrase_ligne = f"Le nombre de personnes supplémentaires sur la ligne {numero_ligne} en raison des JO est {nb_ligne}."

print(phrase_station)
print(phrase_ligne)

NameError: name 'new_horaires_début_df' is not defined

Programme calculant le nombre total de personnes 

In [ ]:
def info_trafic(date, heure, station, ligne):
    try:
        # Calculate nb_station and affluence_horaire
        nb_station, nb_ligne = calculer_personnes_supplementaires(f"{date} {heure}", ligne, station)
        affluence_horaire, _, affluence_ligne = affluence_hors_jo(heure, station, ligne)

        # Calculate the total number of people at the station and on the line
        total_station = nb_station + affluence_horaire

        # Calculate the total number of people on the line
        total_ligne = nb_ligne + affluence_ligne

        return total_station, total_ligne

    except ValueError as e:
        print(e)
        return None, None  # Handle the error as needed

# Example usage of info_trafic function
horaire_info = "09:35"
station_info = "bastille"
ligne_info = "1"
date_info = "2024-07-28"

result_station, result_ligne = info_trafic(date_info, horaire_info, station_info, ligne_info)
if result_station is not None and result_ligne is not None:
    print(f"Nombre de personnes totales à la station {station_info} sur la ligne {ligne_info}: {result_station}")
    print(f"Nombre de personnes totales sur la ligne {ligne_info}: {result_ligne}")

NameError: name 'new_horaires_début_df' is not defined